In [1]:
import os
import sys
import pandas as pd
import unicodedata
import numpy as np
from nltk.corpus import stopwords
from nltk.help import upenn_tagset
from sklearn.feature_extraction.text import CountVectorizer
import string


from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import nltk.data
from gensim.models import Word2Vec
import gensim as gensim

C:\Users\darin\Anaconda2\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
class Book:
    
    def __init__(self, book):
        self.title = ''
        self.author = ''
        #             self.title = book[8].strip().replace('Title: ','')
        #             self.author = book[1].strip()
        #             self.region = book[4].strip()
        #             self.lifespan = book[2].strip()
        #             self.topics = [topic.strip() for topic in book[5].split(',')]
        #             self.type = book[7]
        #             self.text = "".join(book[11:])
        #             self.clean_text = ""
        #             self.author = book[10 .strip().replace('Author: ','')
        n = len(book)
        lim = 30
        if lim > n:
            lim = n
        # Project Gutenburg has no way to determine front matter,
        # so cut first 5% to avoid it
        start = int(n - 0.95*n)
        for line in book[:lim]:
            if 'Title: ' in line:
                self.title = line.strip().replace('Title: ','')
            if 'Author: ' in line:
                self.author = line.strip().replace('Author: ','')
        # start set to strip off various lengths of front-matter
        temp_text = "".join(book[start:]).decode('utf-8')
        # -19350 to strip off the Project Gutenberg donation requests
        self.text = temp_text[0:len(temp_text) - 19350]
        self.clean_text = ''
        self.meaningful_words = []

    def clean(self):
        # Get rid of line breaks. 
        x = self.text.split("\n")
        x = [i.strip() for i in x]
        self.clean_text = " ".join(x)

        # Lower case.
        self.clean_text = self.clean_text.lower()

        # Remove punctuations
        tbl = dict.fromkeys(i for i in xrange(sys.maxunicode)
                  if unicodedata.category(unichr(i)).startswith('P'))
        self.clean_text = self.clean_text.translate(tbl)

        # Remove spaces.
        self.clean_text = self.clean_text.replace("  "," ")
        while "  " in self.clean_text:
            self.clean_text = self.clean_text.replace("  "," ")
    
    
    def remove_stop_words(self):
        stops = set(stopwords.words("english"))
        words = self.clean_text.split()
        self.meaningful_words = [w for w in words if not w in stops]  


    def __getitem__(self, index):
        if index == 0:
            return self.clean_text
        if index == 1:
            return self.title
        if index == 2:
            return self.author

In [ ]:
# Load the corpus of books from a subdirectory. 
book_corpus = []
author = 'dumas'
for filename in os.listdir('./data/' + author):
    if filename[-3:] == 'txt':
        f = open('./data/' + author + '/' + filename)
        book = Book(f.readlines())
        f.close()
        book_corpus.append(book)
        

In [ ]:
n = int(len(book_corpus)*.4)
train_corpus = [book for book in book_corpus[:len(book_corpus) - n]]
test_corpus = [book for book in book_corpus[len(book_corpus) + 1 - n:]]

In [ ]:
for book in train_corpus:
    book.clean()
    book.remove_stop_words()

for book in test_corpus:
    book.clean()
    book.remove_stop_words()

In [ ]:
train_set = []
for book in train_corpus:
    [train_set.append(word) for word in book.meaningful_words]
    
test_set = []
for book in test_corpus:
    [test_set.append(word) for word in book.meaningful_words]

In [ ]:
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(train_set)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

In [ ]:
len(train_data_features[0])

In [ ]:
vocab = vectorizer.get_feature_names()
print vocab[-25:]

In [ ]:
# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
# for tag, count in zip(vocab, dist):
 #   print count, tag

In [ ]:
print dist[0]

In [ ]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:
# Define a function to split a book into parsed sentences
def book_to_sentences(book, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(book.text.encode("ascii","ignore").strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( book_to_wordlist( raw_sentence, remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

def book_to_wordlist(book_text, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    #  
    #  Decode from UTF-8
#     tbl = dict.fromkeys(i for i in xrange(sys.maxunicode)
#               if unicodedata.category(unichr(i)).startswith('P'))
#     book_text = book.text.translate(tbl)

    #
    # 3. Convert words to lower case and split themstring.decode('utf-8')
    words = book_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return words

In [ ]:
sentences = []  # Initialize an empty list of sentences


print "Parsing sentences from training set"
for book in train_corpus:
    sentences += book_to_sentences(book, tokenizer)
    
    
# print "Parsing sentences from unlabeled set"
# for review in unlabeled_train["review"]:
#     sentences += review_to_sentences(review, tokenizer

In [ ]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, 
                          size=num_features, min_count = min_word_count, 
                          window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
# model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = str(num_features)+'features_'+str(min_word_count)+'min_word_count_'+str(context)+'context.npy'
model.save('data/'+model_name)

In [ ]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    print words[:25]
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word != None:
            if word in index2word_set:
                nwords = nwords + 1.
                featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec, nwords)
    return featureVec


def getAvgFeatureVecs(books, model, num_features):
    # Given a set of books (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    bookFeatureVecs = np.zeros((len(books),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for book in books:
        # Print a status message every other book
        print "Book %d of %d" % (counter, len(books))
        # 
        # Call the function (defined above) that makes average feature vectors
        bookFeatureVecs[counter] = makeFeatureVec(book.clean_text.split(), model, num_features)
        #
        # Increment the counter
        counter = counter + 1
    return bookFeatureVecs

In [ ]:
getAvgFeatureVecs(book_corpus, model, num_features)

In [ ]:
import random

def rewrite_my_text(my_text, model):
    wordlist = model.wv.index2word
    punctuation = '.?!",;:()'.split()
    for mark in punctuation:
        my_text = my_text.replace(mark, ' ' + mark)
    lst = my_text.split()
    lst_new = []
    for word in lst:
        if (len(word) > 3) and (word in wordlist):
            word = word.strip()
            lst_new.append(model.most_similar(word)[0][0])
        else:
            lst_new.append(word)
    ret = ' '.join(lst_new)
    for mark in punctuation:
        ret = ret.replace(' ' + mark, mark)
    return ret

In [ ]:
MyText = """

"""

In [ ]:
# For fun
rewrite_my_text(MyText, model)

In [ ]:
#upenn_tagset()


In [ ]:
from nltk import pos_tag, word_tokenize
tagged_words = pos_tag(book_corpus[0].clean_text.split())

In [ ]:
tagged_words[0]

In [ ]:
type(tagged_words[0])

In [2]:
Google_model = gensim.models.KeyedVectors.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)

In [3]:
simple_english_df = pd.read_csv('data/simple_english_wordlist.csv')
simple_english = [a for a in simple_english_df['a']]
simple_english.append('a')
simple_english.append('an')
simple_english.append('is')

In [4]:
import cPickle as pickle
from nltk.stem.lancaster import LancasterStemmer



try:
    my_dict = pickle.load(open('data/simple_english_translations.pickle', "rb" ))
except:
    st = LancasterStemmer()
    stem_se = [st.stem(word) for word in simple_english]
    print 'stem_se...done'
    vocab_google = Google_model.vocab.keys()
    stem_gn = [st.stem(key) for key in Google_model.vocab.keys()]
    print 'stem_gn...done'
    my_dict = {}
    simple_english = list(pd.read_csv('data/simple_english_wordlist.csv', header=None)[0])
    threshold = 0.4
    for sim_in in xrange(len(simple_english)):
        my_dict[simple_english[sim_in]] = simple_english[sim_in]
        print
        print simple_english[sim_in]
        print '**'*8
        indices = [i for i, s in enumerate(stem_gn) if stem_se[sim_in] == s]
        check = [i for i, s in enumerate(vocab_google) if simple_english[sim_in] == s]
        #print check, indices
        if len(check) > 0:
            for index in indices: 
                if Google_model.similarity(simple_english[sim_in], vocab_google[index]) >= threshold:
                    print '{} -> {}'.format(vocab_google[index], Google_model.similarity(simple_english[sim_in], vocab_google[index]))
                    my_dict[vocab_google[index]] = vocab_google[index]
    with open('data/simple_english_translations.pickle', 'wb') as handle:
         pickle.dump(my_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# upenn_tagset()
print 'hi'

In [5]:
import time as time
import sys 
from nltk import pos_tag, word_tokenize
from math import sqrt


def make_simple_english(input_text, threshold=0):
    '''
    Return the input_text translated into simple english
    Input: String
    Output: String
    '''
    if threshold == 0:
        threshold = 60.0/sqrt(len(input_text))
    done = 0
    # timer...
    start= time.clock()
    input_text = input_text.replace('—',' - ').replace("’"," ' ")
    input_text = ''.join([a if ord(a) < 128 else '' for a in list(input_text)])
    words = pos_tag(word_tokenize(input_text))
    #print words
    # These simply pass thru the model

    pass_thru = ['CD', # CD: numeral, cardinal
                 'EX', # EX: existential there
                 'FW', # FW: foreign word
                 'LS', # LS: list item marker
                 'NNP', # NNP: noun, proper, singular
                 'NNPS', # NNPS: noun, proper, plural
                 'PRP', # PRP: pronoun, personal
                 'SYM', # SYM: symbol
                 'TO', # TO: "to" as preposition or infinitive marker
                 'POS',
                 '$', # $: dollar
                 '(',
                 ')',
                 ',',
                 '.',
                 '::',
                 '"'
                ] 
    # make these simple
    make_simple = ['CC', # CC: conjunction, coordinating
                   'DT', # DT: determiner
                   'IN', # IN: preposition or conjunction, subordinating
                   'JJ', # JJ: adjective or numeral, ordinal
                   'JJR', # JJR: adjective, comparative
                   'JJS', # JJR: adjective, comparative
                   'MD', # MD: modal auxiliary
                   'NN', # NN: noun, common, singular or mass
                   'NNS', # NNS: noun, common, plural
                   'PDT', # PDT: pre-determiner
                   'PDT', # PDT: pre-determiner
                   'PRP$', # PRP$: pronoun, possessive
                   'RB', # RB: adverb
                   'RBR', # RBR: adverb, comparative
                   'RBS', # RBS: adverb, superlative
                   'RP', # RP: particle
                   'UH', # UH: interjection
                   'VB', # VB: verb, base form
                   'VBD', # VBD: verb, past tense
                   'VBG', # VBG: verb, present participle or gerund
                   'VBN', # VBN: verb, past participle
                   'VBP', # VBP: verb, present tense, not 3rd person singular
                   'VBZ', # VBZ: verb, present tense, 3rd person singular
                   'WDT', # WDT: WH-determiner
                   'WP', # WP: WH-pronoun
                   'WP$', # WP$: WH-pronoun, possessive
                   'WRB' #WRB: Wh-adverb
                  ]
    done == 0
    count_replacements = 0
    lst_ret = []
    for word in words:
        if word[1] in pass_thru:
            # put it in and move on...
            lst_ret.append(word[0])
        else:
            # We have a word to replace...
            clean = word[0].strip(string.punctuation).lower()
            if clean in new_dict.keys():
                # already simple...
                lst_ret.append(retain_capitalization(new_dict[clean], word[0]))
            else:
                # print '*********************'
                # print 'New word: {}'.format(word)
                # NOT simple english...
                start_this = time.clock()
                try:
                    lst = list(set(Google_model.most_similar(clean)))
                    done = 0
                    n = 0
                    while done == 0:
                        check = list(lst)[n][0]
                        n +=1
                        check_clean = check.strip(string.punctuation).lower()
                        if check_clean in new_dict.keys():
                            done = 1
                            # add to dictionary...based on what's there
                            new_dict[clean] = new_dict[check_clean]
                            # add to lst
                            lst_ret.append(retain_capitalization(new_dict[clean], word[0]))
                            print "     {}: {} -> {} ({}s) {}".format(word, clean, check_clean, time.clock()-start_this, len(lst))
                        else:
                            # add all similar words to that to the lst
                            if time.clock() - start_this < threshold:
                                [lst.append(a) for a in Google_model.most_similar(check, topn=3) if a not in lst]
                            else:
                                done = 1
                                lst_ret.append(retain_capitalization(new_dict[clean], word[0]))
                                new_dict[clean] = new_dict[clean]
                                print "     {}: {} -> {} ({}s) {}".format(word, clean, check_clean, time.clock()-start_this, len(lst))         
                except:
                    lst_ret.append(retain_capitalization(word[0], word[0]))
                    new_dict[word[0].lower()] = word[0].lower()
                    print "     >{}: {} [->] {} ({}s)".format(word, clean, word[0], time.clock()-start_this)
#     if done == 1:
#         for key in new_dict.keys():
#             my_dict[key] = new_dict[key]
    end = time.clock()
    print 'Time: {:.2f}s'.format(end-start)
    txt = replace_punctuation(' '.join(lst_ret))
    txt = txt.encode('utf-8')
    txt = re.sub("\xe2\x80\x93", "-", txt)
    return txt


def retain_capitalization(new_word, original_word):
    '''
    Checks the original_word for capitalization, if it has it, capitalizes the frst letter
    of new_word, returns new_word.
    '''
    if original_word[0] in list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
        lst = list(new_word)
        lst[0] = lst[0].upper()
        new_word = ''.join(lst)
    return new_word


def replace_punctuation(text):
    '''
    Tokenizing takes the punctuation as it's own item in the list.
    This takes the created string and replaces all 'end ?' with 'end?'
    '''
    text = text.replace(' .','.')
    text = text.replace(' ?','?')
    text = text.replace(' !','!')
    text = text.replace(' ,',',')
    text = text.replace(' ;',';')
    text = text.replace(' "','"')
    text = text.replace(" '","'")
    text = text.replace('( ','(')
    text = text.replace(' )',')')
    text = text.replace('$ ','$')
    text = text.replace(' *','*')
    return text

In [10]:
#print new_text
new_dict = my_dict.copy()

In [11]:
MyText = """
Reading is a way of getting information from something that is written. Reading involves recognising the symbols that make up a language. Reading and hearing are the two most common ways to get information. Information gained from reading can include entertainment, especially when reading fiction or humor.

Reading by people is mostly done from paper. Stone, or chalk on a blackboard, or ink on a whiteboard can also be read. Computer displays can be read.

Reading can be something that someone does by themself or they can read aloud. This could be done to benefit other listeners. It could also be done to help concentrate.

Proofreading is a kind of reading that is done to find mistakes in 

"""

In [12]:
new_text = make_simple_english(MyText, 300.0)

     ('statistics', 'NNS'): statistics -> numbers (0.924537373388s) 10
     >('linear', 'JJ'): linear [->] linear (300.161305488s)
     >('regression', 'NN'): regression [->] regression (300.704998671s)
     ('approach', 'NN'): approach -> pragmatic (7.28060648224s) 34
     >('modeling', 'VBG'): modeling [->] modeling (300.784633539s)
     >('scalar', 'JJ'): scalar [->] scalar (300.661919216s)
     >('variable', 'JJ'): variable [->] variable (300.529204311s)
     >('y', 'NN'): y [->] y (273.527993448s)
     ('explanatory', 'JJ'): explanatory -> examples (20.4347001268s) 76
     >('variables', 'NNS'): variables [->] variables (300.255357327s)
     ('independent', 'JJ'): independent -> independently (4.94004550316s) 25
     ('denoted', 'VBD'): denoted -> refers (35.688375678s) 73
     ('multiple', 'JJ'): multiple -> numerous (5.64551469174s) 28
     >('[', 'CC'):  [->] [ (0.000106666540432s)
     >(']', 'NN'):  [->] ] (0.000117333194339s)
     ('distinct', 'JJ'): distinct -> unique (0.82

In [ ]:
print new_text

In [ ]:
mykeys = my_dict.keys()
l1 = len(mykeys)
l2 = len(new_dict.keys())
for key in new_dict.keys():
    if key not in mykeys:
        print "my_dict['{}'] = '{}'".format(key, new_dict[key])



In [ ]:
my_dict['approach'] = 'approach'



In [ ]:

with open('data/simple_english_translations.pickle', 'wb') as handle:
       pickle.dump(my_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# this process was used to create the dictionary for my simple english words...

stem_se

In [ ]:
# for i, word in enumerate(simple_english):
#     print '{}->{}'.format(word, stem_se[i])

In [ ]:
# with open('data/simple_english_translations.pickle', 'wb') as handle:
#       pickle.dump(my_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
for idx in xrange(1, 101):
    ret = ''
    if idx % 3 == 0:
        ret = 'Fizz'
    if idx % 5 == 0:
        ret = ret+'Buzz'
    if ret == '':
        print str(idx)
    else:
        print ret

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz
16
17
Fizz
19
Buzz
Fizz
22
23
Fizz
Buzz
26
Fizz
28
29
FizzBuzz
31
32
Fizz
34
Buzz
Fizz
37
38
Fizz
Buzz
41
Fizz
43
44
FizzBuzz
46
47
Fizz
49
Buzz
Fizz
52
53
Fizz
Buzz
56
Fizz
58
59
FizzBuzz
61
62
Fizz
64
Buzz
Fizz
67
68
Fizz
Buzz
71
Fizz
73
74
FizzBuzz
76
77
Fizz
79
Buzz
Fizz
82
83
Fizz
Buzz
86
Fizz
88
89
FizzBuzz
91
92
Fizz
94
Buzz
Fizz
97
98
Fizz
Buzz
